In [1]:
# chnage the working directory to point to the root directory
import os

os.chdir("../")

In [2]:
# imports
import numpy as np
import pandas as pd

In [3]:

# imports for plots
from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"

In [4]:
# polygon points 
point_bottom_left = (21.30230225738808, -18.36321420540108)
point_bottom_right = (21.528764710648684, -12.594523742277499)

point_center_right = (32.11065951756146, -0.9222339741285166)
point_top_right = (35.11411895680675, -1.4443331500760772)

point_top_left = (36.499784004247495, -7.567132577097473)

In [5]:
SATELLAITE_RESOLUTION = 0.3

# grid points
bottom_left = (21.191028281152967, -17.77179828879979)
bottom_right = (20.969592616916337, 0.07450081631682927)

top_right = (36.80441389073039, 0.9288449224128301)
top_left = (36.766401087897854, -16.490282129655775)

# nb points in each direction
nb_points_lat = int((top_left[0] - bottom_left[0]) / SATELLAITE_RESOLUTION) + 1
nb_points_lon = int((top_right[1] - top_left[1]) / SATELLAITE_RESOLUTION) + 1

# generate grid from bottom to top
# and from left to right
grid_points = []

for i in range(nb_points_lat):
    for j in range(nb_points_lon):
        point = (bottom_left[0] + i * SATELLAITE_RESOLUTION, bottom_left[1] + j * SATELLAITE_RESOLUTION)

        grid_points.append(point)

In [3]:
# plot grid

lat = [point[0] for point in grid_points]
lon = [point[1] for point in grid_points]

fig = go.Figure(data=[
    go.Scattergeo(lat=lat, lon=lon, mode="markers")
])

fig.show(renderer="notebook")

In [7]:
print(len(lat))

3068


In [8]:
# preselction of points
# filter points
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

polygon = Polygon([point_bottom_left, point_bottom_right, point_center_right, point_top_right, point_top_left])

selected_points = []

for point in grid_points:
    p = Point(point)
    
    if polygon.contains(p):
        selected_points.append(point)

In [2]:
# check filtering
lat = [point[0] for point in selected_points]
lon = [point[1] for point in selected_points]

fig = go.Figure(data=[
    go.Scattergeo(lat=lat, lon=lon, mode="markers")
])

fig.show(renderer="notebook")

In [10]:
len(lat)

1247

In [11]:
# determine the country to which a coordinates belongs
import requests


def get_region_info(lat, lon):
    # request params
    ACCESS_KEY = "12d5dd3bb61e853acf1fa318137b85dd"
    lat, lon = lat, lon
    URL = f"http://api.positionstack.com/v1/reverse?access_key={ACCESS_KEY}&query={lat},{lon}"

    # request server
    try:
        res = requests.get(URL).json()
        country, region, name = res["data"][0]["country"], res["data"][0]["region"], res["data"][0]["name"]
    except:
        return "None", "None", "None" 

    return country, region, name

In [12]:
# filter points that belongs to morocco
countries = ['Morocco', 'Western Sahara']
morocco_regions = pd.DataFrame(columns=["lat", "lon", "region", "name"])

for point in selected_points:
    lat, lon = point
    country, region, name = get_region_info(lat, lon)

    # add point whose country is morocco
    if country in countries:
        frame = {
            "lat": [lat],
            "lon": [lon],
            "region": [region],
            "name": [name]
        }

        morocco_regions = morocco_regions.append(pd.DataFrame(frame), ignore_index=True)

In [1]:
fig = px.scatter_geo(morocco_regions, lat="lat", lon="lon")

fig.show(renderer="notebook")

In [14]:
morocco_regions

,lat,lon,region,name
0,21.491028,-16.871798,Oued Ed-Dahab,Oued Ed-Dahab
1,21.491028,-16.571798,Oued Ed-Dahab,Oued Ed-Dahab
2,21.491028,-16.271798,Oued Ed-Dahab,Oued Ed-Dahab
3,21.491028,-15.971798,Western Sahara,Western Sahara
4,21.491028,-15.671798,Oued Ed-Dahab,Oued Ed-Dahab
...,...,...,...,...
700,35.291028,-3.671798,Oriental,Kilati
701,35.291028,-3.071798,Oriental,Playa Los Piratos
702,35.591028,-5.771798,Tangier-Tetouan,Talae Dahnoun
703,35.591028,-5.471798,Tangier-Tetouan,Bounazzal 1


In [17]:
# save data frame
# morocco_regions.to_csv("raw_data/moroccan_regions.csv", index=False)